In [1]:
import numpy as np
import pandas as pd
import torch

In [17]:
n = 2000
X = np.random.rand(n, 424)
y = np.random.rand(n)

def LSTM_sample(X, y, interval, lag = 2):
    # 自变量X从二维矩阵转化为适合LSTM的三维格式
    sample_n = len(y)
    LSTM_X, LSTM_y = [], []
    for i in range(2*interval, sample_n):
        lag_list = []
        for j in range(lag):
            lag_list.append(i-j*interval)
        LSTM_X.append(X[lag_list[::-1],:])
        LSTM_y.append(y[i])
    return np.array(LSTM_X), np.array(LSTM_y)

train_X, train_y = LSTM_sample(X, y, interval = 3, lag = 3)
print(train_X.shape, train_y.shape)

(1994, 3, 424) (1994,)


In [18]:
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
class MyDataset(Dataset):
    def __init__(self, x, y):
        super(MyDataset, self).__init__()
        assert x.shape[0] == y.shape[0] # assuming shape[0] = dataset size
        self.x = x
        self.y = y

    def __len__(self):
        return self.y.shape[0]
    def __getitem__(self, index):
        return self.x[index], self.y[index]

LSTM_traindata = MyDataset(train_X, train_y)
LSTM_trainloader = torch.utils.data.DataLoader(LSTM_traindata, batch_size = 128, shuffle=False)

In [19]:
for batch, (inputs, output) in enumerate(LSTM_trainloader):
    print(inputs.shape)

torch.Size([128, 3, 424])
torch.Size([128, 3, 424])
torch.Size([128, 3, 424])
torch.Size([128, 3, 424])
torch.Size([128, 3, 424])
torch.Size([128, 3, 424])
torch.Size([128, 3, 424])
torch.Size([128, 3, 424])
torch.Size([128, 3, 424])
torch.Size([128, 3, 424])
torch.Size([128, 3, 424])
torch.Size([128, 3, 424])
torch.Size([128, 3, 424])
torch.Size([128, 3, 424])
torch.Size([128, 3, 424])
torch.Size([74, 3, 424])


In [38]:
from torch import nn
class LSTM(nn.Module):
 
    def __init__(self, input_dim = 424, hidden_dim = 200, batch_size = 128, output_dim = 1, num_layers = 5):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
 
        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first = True)
 
        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)
         
    def init_hidden(self):
        self.hidden_cell = (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim), 
                            torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
 
    def forward(self, input):
        lstm_out, self.hidden_cell = self.lstm(input, self.hidden_cell)
        print(lstm_out.shape)
        y_pred = self.linear(lstm_out[:,-1,:])
        return y_pred.view(-1)
model = LSTM()

In [39]:
for batch, (inputs, output) in enumerate(LSTM_trainloader):
    inputs = Variable(inputs).float()
    model.init_hidden()
    model(inputs)

torch.Size([128, 3, 200])
torch.Size([128, 3, 200])
torch.Size([128, 3, 200])
torch.Size([128, 3, 200])
torch.Size([128, 3, 200])
torch.Size([128, 3, 200])
torch.Size([128, 3, 200])
torch.Size([128, 3, 200])
torch.Size([128, 3, 200])
torch.Size([128, 3, 200])
torch.Size([128, 3, 200])
torch.Size([128, 3, 200])
torch.Size([128, 3, 200])
torch.Size([128, 3, 200])
torch.Size([128, 3, 200])


RuntimeError: Expected hidden[0] size (5, 74, 200), got (5, 128, 200)